# commom setting

In [ ]:
import sys
import pandas as pd
import os
from glob import glob
sys.path.append("../input/timmmaster/")
sys.path.append("../input/pythonbox")

In [ ]:
test_df=pd.read_csv('../input/petfinder-pawpularity-score/test.csv')
test_df["Jpg"] = test_df["Id"].apply(lambda x: os.path.join('../input/petfinder-pawpularity-score/test', x + ".jpg"))

In [ ]:
#store predictions of each model
model_predictions=[]

# Beit-L-224

In [ ]:
from timm import create_model

import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_error
import gc
from fastai.vision.all import *

MODEL_WEIGHT='../input/exp044-044-beit-large-chaug-mixup-chbs'
PRETRAINED='beit_large_patch16_224'
IMG_SIZE = 224
N_FOLDS = 10
BATCH_SIZE = 32
SEED = 2021

set_seed(SEED, reproducible=True)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
torch.use_deterministic_algorithms = True

In [ ]:
dataset_path = Path('../input/petfinder-pawpularity-score/')
dataset_path.ls()

In [ ]:
df = pd.read_csv(f'{dataset_path}/train.csv')
df["Jpg"] = df["Id"].apply(lambda x: os.path.join(f'{dataset_path}/train', x + ".jpg"))
df['norm_score'] = df['Pawpularity']/100

In [ ]:
all_preds = []

for i in range(N_FOLDS):   

    print(f'Fold {i} results')
    
    dls = ImageDataLoaders.from_df(df, #pass in train DataFrame
                               valid_pct=0.2, #80-20 train-validation random split
                               path='.',
                            #    folder='train',
                               seed=SEED, #seed
                               fn_col='Jpg', #filename/path is in the second column of the DataFrame
                               label_col='norm_score', #label is in the first column of the DataFrame
                               y_block=RegressionBlock, #The type of target
                               bs=BATCH_SIZE, #pass in batch size
                               num_workers=8,
                               item_tfms=Resize(IMG_SIZE), #pass in item_tfms→バッチ化前の各アイテムに適用
                               batch_tfms=setup_aug_tfms([Flip(),])) #各バッチに適用
    
    #batch_sizeは、作成元のデータローダーの値を引き継ぐ
    test_dl=dls.test_dl(test_df)
#     break
    
    model = create_model(PRETRAINED, pretrained=False, num_classes=dls.c)
    learn = Learner(
        dls, 
        model, 
        model_dir= MODEL_WEIGHT,
        loss_func= BCEWithLogitsLossFlat()
    ).to_fp16()
    
    # learnerの{self.path}/{self.model_dir}/{file}.pthから読み込み
    learn.load(f'{i+1}fold_best')
    
    
    preds, _ = learn.tta(dl=test_dl, n=3, beta=0.5)
    
    all_preds.append(preds)
    
    del learn

    torch.cuda.empty_cache()

    gc.collect()


In [ ]:
preds = (np.mean(np.hstack(all_preds), axis=1))*100
model_predictions.append(preds)

# Swin-L-224

In [ ]:
from timm import create_model

import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_error
import gc
from fastai.vision.all import *

MODEL_WEIGHT='../input/exp056-056-swin-large-224-new'
PRETRAINED = 'swin_large_patch4_window7_224'
IMG_SIZE = 224
N_FOLDS = 10
BATCH_SIZE = 32

SEED=1
set_seed(SEED, reproducible=True)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
torch.use_deterministic_algorithms = True


def petfinder_rmse(input,target):
    return torch.sqrt(F.mse_loss(100*F.sigmoid(input.flatten()), 100*target))

In [ ]:
dataset_path = Path('../input/petfinder-pawpularity-score/')
dataset_path.ls()

In [ ]:
df = pd.read_csv(f'{dataset_path}/train.csv')
df["Jpg"] = df["Id"].apply(lambda x: os.path.join(f'{dataset_path}/train', x + ".jpg"))
df['norm_score'] = df['Pawpularity']/100

In [ ]:
all_preds = []

for i in range(N_FOLDS):   

    print(f'Fold {i} results')
    
    dls = ImageDataLoaders.from_df(df, #pass in train DataFrame
                               valid_pct=0.2, #80-20 train-validation random split
                               path='.',
                            #    folder='train',
                               seed=SEED, #seed
                               fn_col='Jpg', #filename/path is in the second column of the DataFrame
                               label_col='norm_score', #label is in the first column of the DataFrame
                               y_block=RegressionBlock, #The type of target
                               bs=BATCH_SIZE, #pass in batch size
                               num_workers=8,
                               item_tfms=Resize(IMG_SIZE), #pass in item_tfms→バッチ化前の各アイテムに適用
                               batch_tfms=setup_aug_tfms([Flip(),])) #各バッチに適用
    
    #batch_sizeは、作成元のデータローダーの値を引き継ぐ
    test_dl=dls.test_dl(test_df)
#     break
    
    model = create_model(PRETRAINED, pretrained=False, num_classes=dls.c)
    learn = Learner(
        dls, 
        model, 
        model_dir= MODEL_WEIGHT,
        loss_func= BCEWithLogitsLossFlat()
    ).to_fp16()
    
    # learnerの{self.path}/{self.model_dir}/{file}.pthから読み込み
    learn.load(f'{i+1}fold_best')
    
    
    preds, _ = learn.tta(dl=test_dl, n=4, beta=0.5)
    
    all_preds.append(preds)
    
    del learn

    torch.cuda.empty_cache()

    gc.collect()


In [ ]:
preds = (np.mean(np.hstack(all_preds), axis=1))*100
model_predictions.append(preds)

# Swin-L-384

In [ ]:
import os
import warnings
from pprint import pprint
from glob import glob
from tqdm import tqdm
import random
import  time
import gc

import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torchvision.transforms as T
from box import Box
from timm import create_model
from sklearn.model_selection import StratifiedKFold
from torchvision.io import read_image
from torch.utils.data import DataLoader, Dataset

#RAPIDS
import cupy as cp
import cuml, pickle
from cuml.svm import SVR

warnings.filterwarnings("ignore")


In [ ]:
def sigmoid(a):
    return 1 / (1 + np.exp(-a))

In [ ]:
def set_seed(seed):
    np.random.seed(seed)
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

set_seed(2021)

In [ ]:
dense_features = [
        'Subject Focus', 'Eyes', 'Face', 'Near', 'Action', 'Accessory',
        'Group', 'Collage', 'Human', 'Occlusion', 'Info', 'Blur'
    ]

class PetfinderDataset(Dataset):
    #image_sizeはresizeで用いるsize
    def __init__(self, df, image_size=224):
        self._X = df["Jpg"].values
        self._features=df[dense_features].values
        self._y = None
        #カラムの一覧を取得
        if "Pawpularity" in df.keys():
            self._y = df["Pawpularity"].values
        self._transform = T.Resize([image_size, image_size])

    def __len__(self):
        return len(self._X)

    def __getitem__(self, idx):
        image_path = self._X[idx]
        fearure = self._features[idx]
        #rgbの３チャンネルを含んだtensorに変換
        image = read_image(image_path)
        image = self._transform(image)
        if self._y is not None:
            label = self._y[idx]
            return image, fearure, label
        return image, fearure

In [ ]:
class Model(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.cfg = cfg
        self.__build_model()
        self.transforms = get_default_transforms()
        

    def __build_model(self):
        self.backbone = create_model(
            self.cfg.model.name, pretrained=False, num_classes=0, in_chans=3
        )
        num_features = self.backbone.num_features
        self.fc = nn.Sequential(
            nn.Dropout(0.5), nn.Linear(num_features+12, self.cfg.model.output_dim)
        )

    def forward(self, x1, x2, svr_path, tta=True, svr_weight=0.45):
        svr_head = pickle.load(open(svr_path, "rb"))
        
        if tta == True:
            nn_preds=[]
            svr_preds=[]
            for k,transform in self.transforms.items():
                x=transform(x1)
                x = self.backbone(x)
                x = torch.cat([x, x2], dim=1)
                #tensor
                nn_pred = self.fc(x)
                #rapidのndarray
                svr_pred = svr_head.predict(x)
                
                nn_preds.append(nn_pred)
                svr_preds.append(svr_pred)
            
            nn_pred=torch.mean(torch.stack(nn_preds, dim=0), dim=0)
            nn_pred=nn_pred.to('cpu').detach().numpy()
            nn_pred=np.array([sigmoid(p)*100 for p in nn_pred]).ravel()
            
            #ndarrayに変換してからmean
            svr_pred = np.mean(cp.array(svr_preds).get(), axis=0)
            
            pred=(1-svr_weight)*nn_pred+svr_weight*svr_pred

        return pred


In [ ]:
#weight_pathとimage_sizeと、model_nameは変更
config = {'seed': 2021,
          'root': '/kaggle/input/petfinder-pawpularity-score/', 
          'weight_path':'../input/exp006-tta1',
          'svr_path': '../input/svrhead-for-exp006',
          'transform':{
              'name': 'get_default_transforms',
              'image_size': 384
          },
          'test_loader': {
              'batch_size': 32,
              'shuffle': False,
              'num_workers': 4,
              'pin_memory': False,
              'drop_last': False
         },
          'model':{
              'name': 'swin_large_patch4_window12_384',
              'output_dim': 1
          },
          'device':'cuda' if torch.cuda.is_available() else 'cpu',
          'amp':True,
          'tta':True
}

config = Box(config)

In [ ]:
IMAGENET_MEAN = [0.485, 0.456, 0.406]  # RGB
IMAGENET_STD = [0.229, 0.224, 0.225]  # RGB


#for TTA
def get_default_transforms(image_size: int = config.transform.image_size):

    transforms_dict = {
        "default": T.Compose(
            [
                T.ConvertImageDtype(torch.float),
                T.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD),
            ]
    ),
        "tta_flip": T.Compose(
            [   
                T.RandomHorizontalFlip(p=1),
                T.ConvertImageDtype(torch.float),
                T.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD),
            ]
    ),
        "tta_rotate": T.Compose(
            [   
                T.RandomRotation(degrees=45),
                T.ConvertImageDtype(torch.float),
                T.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD),
            ]
    )
    }

    return transforms_dict

In [ ]:
test_dataset=PetfinderDataset(test_df, config.transform.image_size)
test_dataloader=DataLoader(test_dataset,**config.test_loader)

In [ ]:
model = Model(config) 
model.to(config.device)
model_paths=glob(f'{config.weight_path}/*')

In [ ]:
fold_predictions=[]
for fold in range(10):
    
    model_weight = f'../input/exp006-tta1/{fold+1}fold_best_loss.ckpt'
    svr_path=f'{config.svr_path}/SVR_fold_{fold}.pkl'
    
    print(model_weight)
    print(svr_path)
    model.load_state_dict(torch.load(model_weight)['state_dict'])
    model.eval()
    
    predictions=[]
    with torch.no_grad():
        #半精度
        if config.amp==True:
            with torch.cuda.amp.autocast(enabled=True):
                for img, features in test_dataloader:
                    img=img.to(config.device)
                    features=features.to(config.device)
                    out=model(img, features, svr_path, tta=config.tta, svr_weight=0.45)
                    predictions.append(out)
        
        else:
            for img, features in test_dataloader:
                img=img.to(config.device)
                features=features.to(config.device)
                out=model(img, features, svr_path, tta=config.tta, svr_weight=0.45)
                predictions.append(out)
                
    predictions=np.hstack(predictions)
    
    fold_predictions.append(predictions)

In [ ]:
final_predictions=np.mean(fold_predictions,axis=0)
model_predictions.append(final_predictions)

In [ ]:
del test_dataset,test_dataloader,model
gc.collect()
torch.cuda.empty_cache()

# weight_ensemble

In [ ]:
w1=0.36826336
w2=0.35625861
w3=0.27547803
ens_pred=w1*model_predictions[0]+w2*model_predictions[1]+w3*model_predictions[2]

In [ ]:
test_df["Pawpularity"] = ens_pred
test_df = test_df[["Id", "Pawpularity"]]
test_df.to_csv("submission.csv", index=False)

In [ ]:
test_df.head()